In [ ]:
import json

# 打开并读取JSON文件
with open('dureadqg/train.json', 'r') as file:
    data = json.load(file)

# 输出读取到的数据
print(data)

In [17]:
import json

# 定义一个空列表用于存储所有的字典
train_list = []
test_list = []
# 打开JSON Lines文件
with open('dureadqg/train.json', 'r') as file:
    # 逐行读取
    for line in file:
        # 解析每一行
        data_dict = json.loads(line)
        # 将解析后的字典添加到列表中
        train_list.append(data_dict)
# 打开JSON Lines文件
with open('dureadqg/dev.json', 'r') as file:
    # 逐行读取
    for line in file:
        # 解析每一行
        data_dict = json.loads(line)
        # 将解析后的字典添加到列表中
        test_list.append(data_dict)


In [18]:
print(len(train_list), len(test_list))

14520 984


In [ ]:
!pip install datasets transformers rouge-score nltk

In [19]:
from datasets import *
train_dataset = Dataset.from_list(train_list)
test_dataset = Dataset.from_list(test_list)
print(train_dataset, test_dataset)

Dataset({
    features: ['context', 'answer', 'question', 'id'],
    num_rows: 14520
}) Dataset({
    features: ['context', 'answer', 'question', 'id'],
    num_rows: 984
})


In [1]:
!git lfs install

Updated Git hooks.
Git LFS initialized.


In [2]:
!git clone https://huggingface.co/uer/t5-base-chinese-cluecorpussmall

正克隆到 't5-base-chinese-cluecorpussmall'...
fatal: 无法访问 'https://huggingface.co/uer/t5-base-chinese-cluecorpussmall/'：Failed to connect to huggingface.co port 443: 连接超时


In [3]:
from modelscope.hub.api import HubApi

YOUR_ACCESS_TOKEN = '49834fc2-f98f-4ac3-8efc-01ce1764baf8'

api = HubApi()
api.login(YOUR_ACCESS_TOKEN)
api.push_model(
    model_id="zlyzly/t5-base-chinese-cluecorpussmall", 
    model_dir="t5-base-chinese-cluecorpussmall" # 本地模型目录，要求目录中必须包含configuration.json
)

2024-07-24 10:21:54,294 - modelscope - INFO - [master ee39e78] 'upload model'
 5 files changed, 21385 insertions(+)
 create mode 100644 README.md
 create mode 100644 config.json
 create mode 100644 special_tokens_map.json
 create mode 100644 tokenizer_config.json
 create mode 100644 vocab.txt



In [4]:
from modelscope import HubApi
from modelscope import snapshot_download

api=HubApi()
api.login('49834fc2-f98f-4ac3-8efc-01ce1764baf8')

# download your model, the model_path is downloaded model path.
model_path =snapshot_download(model_id='zlyzly/t5-base-chinese-cluecorpussmall')


Downloading: 100%|██████████| 460/460 [00:00<00:00, 62.7kB/s]
Downloading: 100%|██████████| 26.0/26.0 [00:00<00:00, 3.89kB/s]
Downloading: 100%|█████████▉| 818M/818M [00:03<00:00, 266MB/s]
Downloading: 100%|██████████| 6.32k/6.32k [00:00<00:00, 3.82MB/s]
Downloading: 100%|██████████| 263/263 [00:00<00:00, 178kB/s]
Downloading: 100%|██████████| 108k/108k [00:00<00:00, 28.3MB/s]


In [5]:
model_path

'/mnt/workspace/.cache/modelscope/zlyzly/t5-base-chinese-cluecorpussmall'

In [7]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
tokenizer = AutoTokenizer.from_pretrained(model_path)

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-24 10:24:05.644367: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-24 10:24:05.646044: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-24 10:24:05.681653: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-24 10:24:05.682359: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To 

/opt/conda/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


/opt/conda/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [8]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [9]:
model

T5ForConditionalGeneration(
  (shared): Embedding(21228, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(21228, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro